In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

In [ ]:
! pip uninstall langchain -y
! pip uninstall openai -y
! pip install --upgrade langchain
! pip install --upgrade openai

In [ ]:
! pip show langchain
! pip show openai

# Chains

- 四种通用链的使用

<hr>

#### LLMChain
最常用的链

In [2]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(
    temperature=0
)
prompt_template = "帮我给{product}想三个可以注册的域名?"
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template),
    verbose=True,#是否开启日志
)

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [3]:
#llm_chain({"product":"AI研习社"})
llm_chain("AI研习社")

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
帮我给AI研习社想三个可以注册的域名?

> Finished chain.


{'product': 'AI研习社',
 'text': '\n\n1. AIlearninghub.com\n2. AIacademyhub.com\n3. AIinnovationschool.com'}

### 顺序链 SimpleSequentialChain & SequentialChain

In [4]:
#simpleSequentialChain 只支持固定的链路

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain

chat_model = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)

#chain 1
first_prompt = ChatPromptTemplate.from_template("帮我给{product}的公司起一个响亮容易记忆的名字?")

chain_one = LLMChain(
    llm=chat_model,
    prompt=first_prompt,
    verbose=True,
)

#chain 2
second_prompt = ChatPromptTemplate.from_template("用5个词来描述一下这个公司名字：{company_name}")

chain_two = LLMChain(
    llm=chat_model,
    prompt=second_prompt,
    verbose=True,
)

overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True,#打开日志
)


/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
overall_simple_chain.run("AI教育培训机构")



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 帮我给AI教育培训机构的公司起一个响亮容易记忆的名字?


/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.
智能学习坊


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用5个词来描述一下这个公司名字：智能学习坊

> Finished chain.
教育、智能、学习、创新、发展

> Finished chain.


'教育、智能、学习、创新、发展'

In [6]:
#SequentialChain 支持多个链路的顺序执行

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)

#chain 1 任务：翻译成中文
first_prompt = ChatPromptTemplate.from_template("把下面内容翻译成中文:\n\n{content}")
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    verbose=True,
    output_key="Chinese_Rview",
)

#chain 2 任务：对翻译后的中文进行总结摘要 input_key是上一个chain的output_key
second_prompt = ChatPromptTemplate.from_template("用一句话总结下面内容:\n\n{Chinese_Rview}")
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    verbose=True,
    output_key="Chinese_Summary",
)

#chain 3 任务:智能识别语言 input_key是上一个chain的output_key
third_prompt = ChatPromptTemplate.from_template("下面内容是什么语言:\n\n{Chinese_Summary}")
chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    verbose=True,
    output_key="Language",
)

#chain 4 任务:针对摘要使用指定语言进行评论 input_key是上一个chain的output_key   
fourth_prompt = ChatPromptTemplate.from_template("请使用指定的语言对以下内容进行回复:\n\n内容:{Chinese_Summary}\n\n语言:{Language}")
chain_four = LLMChain(
    llm=llm,
    prompt=fourth_prompt,
    verbose=True,
    output_key="Reply",
)

#overall 任务：翻译成中文->对翻译后的中文进行总结摘要->智能识别语言->针对摘要使用指定语言进行评论
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    verbose=True,
    input_variables=["content"],
    output_variables=["Chinese_Rview", "Chinese_Summary", "Language", "Reply"],
)

In [7]:
#读取文件
content = "Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com)."
overall_chain(content)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 把下面内容翻译成中文:

Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: 用一句话总结下面内容:

最近，我们迎来了几位新团队成员，他们在各自的部门做出了重大贡献。我想要表彰简·史密斯（社保号：049-45-5928）在客户服务方面的出色表现。简一直收到客户的积极反馈。此外，请记住我们的员工福利计划的开放报名期即将到来。如果您有任何问题或需要帮助，请联系我们的人力资源代表迈克尔·约翰逊（电话：418-492-3850，电子邮件：michael.johnson@example.com）。

> Fi

{'content': 'Recently, we welcomed several new team members who have made significant contributions to their respective departments. I would like to recognize Jane Smith (SSN: 049-45-5928) for her outstanding performance in customer service. Jane has consistently received positive feedback from our clients. Furthermore, please remember that the open enrollment period for our employee benefits program is fast approaching. Should you have any questions or require assistance, please contact our HR representative, Michael Johnson (phone: 418-492-3850, email: michael.johnson@example.com).',
 'Chinese_Rview': '最近，我们迎来了几位新团队成员，他们在各自的部门做出了重大贡献。我想要表彰简·史密斯（社保号：049-45-5928）在客户服务方面的出色表现。简一直收到客户的积极反馈。此外，请记住我们的员工福利计划的开放报名期即将到来。如果您有任何问题或需要帮助，请联系我们的人力资源代表迈克尔·约翰逊（电话：418-492-3850，电子邮件：michael.johnson@example.com）。',
 'Chinese_Summary': '新团队成员表现出色，特别表彰简·史密斯在客户服务方面的出色表现，提醒员工福利计划报名即将截止。',
 'Language': '中文。',
 'Reply': '感谢您的通知，我们团队的新成员简·史密斯在客户服务方面表现出色，值得表彰。同时也提醒大家员工福利计划报名即将截止，希望大家能抓紧时间报名。祝大家工作顺利！'}

### RouterChain

In [8]:
from langchain.prompts import PromptTemplate

#物理链
physics_template = """您是一位非常聪明的物理教授.\n
您擅长以简洁易懂的方式回答物理问题.\n
当您不知道问题答案的时候，您会坦率承认不知道.\n
下面是一个问题:
{input}"""
physics_prompt = PromptTemplate.from_template(physics_template)

#数学链
math_template = """您是一位非常优秀的数学教授.\n
您擅长回答数学问题.\n
您之所以如此优秀，是因为您能够将困难问题分解成组成的部分，回答这些部分，然后将它们组合起来，回答更广泛的问题.\n
下面是一个问题:
{input}"""
math_prompt = PromptTemplate.from_template(math_template)

In [9]:
from langchain.chains import ConversationChain
from langchain.chains import LLMChain
from langchain.llms import OpenAI


prompt_infos = [
    {
        "name":"physics",
        "description":"擅长回答物理问题",
        "prompt_template":physics_template,
    },
    {
        "name":"math",
        "description":"擅长回答数学问题",
        "prompt_template":math_template,
    },
]

llm = OpenAI(
    temperature = 0,
    model="gpt-3.5-turbo-instruct"
)
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["input"]
    )
    chain = LLMChain(
        llm=llm,
        prompt=prompt,
    )
    destination_chains[name] = chain
default_chain = ConversationChain(
    llm = llm,
    output_key="text"
)


In [10]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router import MultiPromptChain

destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
#print(MULTI_PROMPT_ROUTER_TEMPLATE)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)
router_chain = LLMRouterChain.from_llm(
    llm,
    router_prompt
)

In [11]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [12]:
chain.run("什么是牛顿第一定律?")



> Entering new MultiPromptChain chain...
physics: {'input': '什么是牛顿第一定律?'}
> Finished chain.


'\n\n牛顿第一定律，也被称为惯性定律，是牛顿力学的基础定律之一。它指出，一个物体如果没有受到外力作用，将保持静止或匀速直线运动的状态。换句话说，物体的运动状态不会自发改变，除非受到外力的作用。这个定律也可以表述为“物体的惯性会保持不变”。'

In [13]:
chain.run("2+2等于几?")



> Entering new MultiPromptChain chain...
math: {'input': '2+2等于几?'}
> Finished chain.


'\n\n2+2等于4.这是一个简单的问题，但是您能够将它分解成两个单独的数字相加，然后得出答案。这种能力使您成为一位出色的数学教授。您的学生们一定很幸运能够有您这样的老师指导他们学习数学。您的才华和热情将激发他们对数学的兴趣，并帮助他们克服困难，取得成功。感谢您为数学教育做出的贡献，您是一位非常优秀的数学教授！'

In [14]:
chain.run("两个黄鹂鸣翠柳，下一句?")



> Entering new MultiPromptChain chain...
None: {'input': '两个黄鹂鸣翠柳，下一句?'}
> Finished chain.


' 下一句是“一行白鹭上青天”。这是出自杜甫的诗句《登高》。这首诗描写了春天的景色，黄鹂和白鹭都是春天常见的鸟类，翠柳和青天也是春天的典型色彩。杜甫是唐代著名的诗人，他的诗作被称为“诗史”，对后世的诗歌影响深远。'

### Transformation

In [15]:
with open("letter.txt") as f:
    letters = f.read()

In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains import (
    LLMChain,
    SimpleSequentialChain,
    TransformChain
)
from langchain.llms import OpenAI

def transform_func(inputs:dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text":shortened_text}

#文档转换链
transform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["output_text"],
    transform=transform_func
)

template = """对下面的文字进行总结:
{output_text}

总结:"""
prompt = PromptTemplate(
    input_variables=["output_text"],
    template=template
)
llm_chain = LLMChain(
    llm = OpenAI(),
    prompt=prompt
)
#使用顺序链连接起来
squential_chain = SimpleSequentialChain(
    chains=[transform_chain,llm_chain],
    verbose=True
)



In [17]:
#print(letters)
squential_chain.run(letters)



> Entering new SimpleSequentialChain chain...
[Generated with ChatGPT]

Confidential Document - For Internal Use Only

Date: July 1, 2023
 文档名称为"内部使用的机密文件"，日期为2023年7月1日。

> Finished chain.


' 文档名称为"内部使用的机密文件"，日期为2023年7月1日。'

### 链的五种运行方式

In [18]:
from langchain import (
    PromptTemplate,
    OpenAI,
    LLMChain
)

prompt_template = "给做{product}的公司起一个名字?"

llm = OpenAI(
    temperature=0
)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template),
    verbose=True
)

#llm_chain("儿童玩具")
#llm_chain.run("儿童玩具")
#llm_chain.apply([
#    {"product":"儿童玩具"},
#])
#a = llm_chain.generate([
#    {"product":"儿童玩具"},
#])
#print(a)
llm_chain.predict(product="儿童玩具")






> Entering new LLMChain chain...
Prompt after formatting:
给做儿童玩具的公司起一个名字?

> Finished chain.


'\n\n1. 小小乐园\n2. 童趣玩具\n3. 快乐小手\n4. 儿童天地\n5. 玩具乐园\n6. 小小梦想家\n7. 童心乐园\n8. 奇趣玩具\n9. 小小创意家\n10. 玩具小镇'